In [1]:
#| default_exp nb_state

In [2]:
#| export
from __future__ import annotations

# Notebook state

> Programmatic access to notebook cells and outputs.



# Prologue

In [3]:
#| export

import inspect
import json
import os
import sys
import urllib.parse
from inspect import Parameter
from pathlib import Path
from typing import Mapping

import anywidget
import fastcore.all as FC
import ipykernel
import traitlets as T
from fastcore.foundation import L
from IPython.display import clear_output
from IPython.display import display
from IPython.display import DisplayHandle
from IPython.display import HTML
from olio.basic import bundle_path


In [4]:
os.environ['DEBUG'] = 'True'

In [5]:
#| export

from bridget.bridge import BridgeBase
from bridget.bridge import HTMXCommander
from bridget.bridge_helpers import bridge_js
from bridget.bridge_helpers import BridgeWidget
from bridget.bridge_helpers import loader_js
from bridget.display_helpers import displaydh
from bridget.display_helpers import NBLogger
from bridget.helpers import kounter
from bridget.helpers import skip
from bridget.nb import NB
from bridget.nb import NBCell
from bridget.widget_helpers import anysource
from bridget.widget_helpers import bundled

In [6]:
from datetime import datetime
from typing import Any

from fastcore.test import *
from fasthtml.xtend import Script
from olio.common import pops_
from olio.common import pops_values_
from olio.common import shorten
from olio.common import shortens
from olio.widget import cleanupwidgets
from rich.console import Console


In [7]:
from bridget.bridge import get_bridge
from bridget.bridge_helpers import debug
from bridget.bridge_helpers import Loader
from bridget.bridge_helpers import notdebug
from bridget.bridge_helpers import observer_scr
from bridget.helpers import bridge_cfg
from bridget.helpers import Singleling


----


In [8]:
#| exporti

DEBUG = os.environ.get('DEBUG', None) == 'True'
BUNDLE_PATH = bundle_path(__name__)
_EMPTY = Parameter.empty
EmptyT = type[_EMPTY]

In [9]:
cprint = (console := Console(width=120)).print

In [10]:
bridge_cfg.auto_show = True
bridge_cfg.auto_id = False
# brd = BridgeBase.instance()
# ldr = await Loader.create(show=True)
bridge = await get_bridge()

In [11]:
# observer_js = BUNDLE_PATH / 'js/observer.js'
# observer_scr = Script(debug(observer_js.read_text()), type='module', id='brd-observer-manager')

# await ldr.aload_links(observer=observer_scr)

----

# Cell id

In [12]:
__cellid__ = ''
__cellsource__ = ''

In [13]:
print(f"{__cellid__!r}\n{__cellsource__!r}")
test_eq(__cellsource__[:20], 'print(f"{__cellid__!')

'vscode-notebook-cell:/Users/vic/dev/repo/project/bridget/nbs/21_nb_state.ipynb#X23sZmlsZQ%3D%3D'
'print(f"{__cellid__!r}\\n{__cellsource__!r}")\ntest_eq(__cellsource__[:20], \'print(f"{__cellid__!\')\n'


In [14]:
def _cellid_from(s): return urllib.parse.unquote(urllib.parse.urlparse(s).fragment)
def _nburi_from(s): return urllib.parse.urlparse(s).netloc + urllib.parse.urlparse(s).path
def cellspec_from(s): return (uri := urllib.parse.urlparse(s)).path, urllib.parse.unquote(uri.fragment)
nburi, cellid = cellspec_from(__cellid__)
nburi, cellid

('/Users/vic/dev/repo/project/bridget/nbs/21_nb_state.ipynb', 'X24sZmlsZQ==')

# show_notebook_state

In [15]:
#| export

NBSTATE_MIME = 'application/x-notebook-state'

def show_notebook_state(options:Mapping={'feedback': True, 'watch': True, 'debug': True}) -> DisplayHandle:
    "Display notebook state with our custom mime type"
    dh = display({NBSTATE_MIME: options}, raw=True, display_id=True)
    return dh  # type: ignore

def update_notebook_state(dh: DisplayHandle|None, options:Mapping|None=None):
    if dh: dh.update({NBSTATE_MIME: {**(options or {}), 'id': kounter('nbstate')}}, 
        raw=True)

def clear_notebook_state(dh: DisplayHandle|None): 
    if dh: dh.update(HTML(''))

In [16]:
dh = show_notebook_state()

Simple visual feedback of notebook state. In the front-end we manage just one notebook state, though you can display multiple vies of it with `show_notebook_state`.

Note that the state data **is** still in the **front-end** (JS-land). This function is just a visual feedback of state changes, but we haven't transferred yet the state to the **kernel** (python-land). For that, see next section.


In [17]:
bridge.logger.display()

In [ ]:
print(18)

After executing above cell, you've noticed a pink flash. We're monitoring state changes in real-time.

Clear the output of the above cell. Open the details and look for cell #18 (if you haven't added any new cells above it). Check that it doesn't have any outputs.

Run the cell again and check to see that it now has outputs.

Create a new cell below this, source or markdown, and write something. Notice that we're updating the state of cell inputs not just outputs.

In [21]:
update_notebook_state(dh, {'feedback': False})

With feedback disabled, the details disclosure in not shown.

In [22]:
update_notebook_state(dh, {'feedback': True})

In [23]:
update_notebook_state(dh, {'debug': False})

`debug` affects javascript dev console logging.

In [22]:
# update_notebook_state(dh, {'watch': False})


`watch` false disables real-time updates. You'll need to call `update_notebook_state` manually to see state changes.

In [ ]:
print(29)

In [24]:
# update_notebook_state(dh)

In [25]:
update_notebook_state(dh, {'debug': True})

In [26]:
clear_notebook_state(dh)

Though we've cleared the feedback output, Bridget is still monitoring and capturing state changes, as you can see in the dev console.

In [27]:
bridge.logger.clear_log()

# NBStateFetcher
> A widget to retrieve notebook state from the front-end.

Simple [AnyWidget](https://anywidget.dev/en/getting-started/) to get the state from the front end and bring it back to python where we really need it (and it should really be imho).



In [27]:
#| export

nbstate_js = BUNDLE_PATH / 'js/nbstate.js'
nbstate_esm = bundled(nbstate_js, bundle=__name__, bundler='copy')

class NBStateFetcher(BridgeWidget):
    _esm = anysource(nbstate_esm(debugger=DEBUG), '''
export default { initialize: ({model}) => { return initializeNBState(model) } };
''')
    feedback = True; watch = True; debug = True;
    ctx_name = T.Unicode('nbstate').tag(sync=True)
    state = T.Instance(NB, default_value=NB())

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.feedback_dh = display(HTML(''), display_id=True, metadata=skip())
        self.updates = []
    
    def close(self):
        if self.feedback_dh: self.feedback_dh.update(HTML(''), metadata=skip())
        self.feedback_dh = None
        super().close()

    @property
    def opts(self): return {'feedback': self.feedback, 'watch': self.watch, 'debug': self.debug}

    def _update_notebook_state(self, req_id):
        if self.feedback_dh and not self._loading: self.feedback_dh.update(
            {NBSTATE_MIME: {**self.opts, 'id': req_id}}, raw=True, metadata=skip())
    def update_notebook_state(self, 
            watch:bool|None=None, feedback:bool|None=None, debug:bool|None=None): 
        if watch is not None: self.watch = watch
        if feedback is not None: self.feedback = feedback
        if debug is not None: self.debug = debug
        return self._update_notebook_state(kounter('nbstate'))

    def update(self, timeout: float|None=None):
        if not self.watch:
            self.update_notebook_state()
            self.send({'cmd': 'get_state'}, timeout=timeout)

    async def aupdate(self, timeout: float=20.0):
        if not self.watch:
            self.update_notebook_state()
            await self.asend({'cmd': 'get_state'}, timeout=timeout)
    
    def setup_notebook_state(self):
        self.feedback_dh.update({NBSTATE_MIME: {**self.opts, 'id': kounter('nbstate'), 'update':'full'}},  # type: ignore
            raw=True, metadata=skip())
    
    def on_info(self, *args, info:str, **kwargs):
        super().on_info(*args, info=info, **kwargs)
        if info == 'loaded':
            self.logger.log('Requesting initial notebook state...')
            self.setup_notebook_state()
        elif info == 'found existing state observers':
            self.logger.log('Found existing state observer.')
    
    def on_state_update(self, *args, state:dict|str, **kwargs):
        d = (json.loads(state) if isinstance(state, str) else state or {})
        nbData = d.get('nbdata', {})
        # self.logger.log(f" watch: {self.watch} feedback: {self.feedback} debug: {self.debug}")
        self.updates.append(d)
        self.logger.log(f"State update - type: {d['type']} ts: {d['timestamp']}")
        if d['type'] == 'state':
            self.logger.log(f"---- #cells: {len(d.get('cells', []))}")
            self.state = NB.fromStateMessage(d)
        else:
            self.state.apply_diffsMessage(d)
            diffs:list[dict] = d.get('changes', [])
            self.logger.log(f"#diffs: {len(diffs)}")
            for i, d in enumerate(diffs):
                self.logger.log(f"---- {i} - cells: {[c['idx'] for c in d['cells']]} added: {[c['idx'] for c in d.get('added', [])]} removed: {d.get('removed', [])}")

In [ ]:
cleanupwidgets('fetcher')

bridge.logger.display()
fetcher = NBStateFetcher(logger=bridge.logger)

In [29]:
bridge.logger.clear_log()

In [ ]:
fetcher.updates[-1]

In [ ]:
fetcher.state.cells[41]

In [ ]:
print(cellspec_from(__cellid__))
this_id = f"this_{kounter('this')}"
_ = displaydh(metadata={'bridge': {'this': this_id}})
# _ = fetcher.send({'cmd': 'get_state'}, timeout=5)

In [ ]:
print(cellspec_from(__cellid__))
this_id = f"this_{kounter('this')}"
# _ = displaydh(metadata={'bridge': {'this': this_id}})
_ = fetcher.send({'cmd': 'update', 'id': this_id}, timeout=5)

In [ ]:
fetcher.state.cells[46]

In [ ]:
print('test5')

In [ ]:
display(HTML('<div>test6</div>'))

In [46]:
fetcher.update_notebook_state()

By default, the widget doesn't monitor the notebook state in real-time (`watch=False`).

Call `update_notebook_state` to retrieve the state once.

In [ ]:
dh = displaydh(HTML('test7'))

`displaydh` (simple wrapper around `display`) triggers a state update.

In [31]:
fetcher.update_notebook_state(watch=True)

You can also switch to real-time updates with `update_notebook_state(watch=True)`.

In [ ]:
print('test8')

In [71]:
cleanupwidgets('fetcher')

In [ ]:
uri = urllib.parse.urlparse(fetcher.state.nbData['notebookUri'])
uri, __vsc_ipynb_file__

In [ ]:
fetcher.state.cells[41]

# NBState

In [33]:
#| export
class NBState(T.HasTraits, FC.GetAttr):
    state = T.Instance(NB, default_value=NB())
    _default = 'state'
    def __init__(self, source: NBStateFetcher|Mapping|None=None):
        if source is None: source = NBStateFetcher(show=True)
        self.source = source

    @property
    def source(self): return self._source
    @source.setter
    def source(self, source: NBStateFetcher|Mapping|None=None):
        if hasattr(self, '_link'): self._link.unlink()
        if isinstance(source, NBStateFetcher): 
            self._link = T.dlink((source, 'state'), (self, 'state'))
        elif isinstance(source, Mapping): self.state = NB(**source)
        else: self.state = NB()
        # try: del self.processor
        # except: pass
        self._source = source

    def update(self):
        if isinstance(self.source, NBStateFetcher): self.source.update_notebook_state()
    
    def this(self: NBState, idx:int|None=None) -> NBCell:...
    
    def __del__(self):
        if hasattr(self, '_link'): self._link.unlink()


In [34]:
state = json.loads(Path('../packages/nbinspect-vscode/outputs.json').read_text('utf-8'))

nb = NBState(state)
# test_eq(nb.state, empty_state())
# test_is(nb.state, fetcher.state)


In [ ]:
display(nb.mds)
nb.codes

In [ ]:
nb = NBState()

In [ ]:
shortens(nb.state.source[42:52], 'r', 80)

In [ ]:
shortens(nb.state.find('class').attrgot('source'), 'r', 80)

In [39]:
# find_me

In [ ]:
#| hide
shortens(nb.state.find('# find_me'), 'r', 80)

In [ ]:
#| hide
nb.state.find('# find_me').filter(lambda x: 'hide' not in x.directives_).attrgot('source')

In [ ]:
# this cell2
nb.update()
nb.state.find('# this cell2').attrgot('source')

# this

In [43]:
#| export

FIRST = -sys.maxsize
LAST = sys.maxsize

def _this(nb):
    this_id = f"this_{kounter('this')}"
    _= displaydh(metadata=skip(this=this_id))
    nb.source.send({'cmd': 'get_state'}, timeout=5)
    clear_output(wait=True)
    this_cell = nb.state.find(this_id, 'outputs.0.metadata.bridge.this')[0]
    this_idx = nb.cells.index(this_cell)
    return this_idx, this_cell

@FC.patch
def this(self: NBState, idx:int|None=None) -> NBCell:
    "Current cell if `idx` is None, or cell at `idx` from current cell upwards. Raises if not found."
    if not isinstance(self.source, NBStateFetcher): return self.cells[idx or -1]
    this_idx, this_cell = _this(self)
    if not idx: return this_cell
    return self.cells[max(0, this_idx + idx)] if idx < 0 else self.cells[min(len(self.cells)-1, this_idx + idx)]

@FC.patch
def these(self: NBState, above:int|None=None, below:int|None=None) -> L:
    if not isinstance(self.source, NBStateFetcher): return self.cells[above or 0:below or None]
    this_idx, _ = _this(self)
    return self.cells[max(0, this_idx + (above or FIRST)):min(len(self.cells), this_idx+1+(below or 0))]


In [ ]:
# this cell
cell = nb.this()
cell.source

In [ ]:
cell = nb.this(-1)
cell.source

In [ ]:
cell = nb.this(-3)
cell.source

In [ ]:
cell = nb.this(FIRST)
cell.source

In [ ]:
cell = nb.this(1)
cell.source

# Bridge

In [48]:
#| export

@FC.delegates()
class Bridge(BridgeBase):
    _esm = anysource('debugger;' if DEBUG else '', bridge_js, loader_js, nbstate_js,'''
export default { 
    initialize({ model }) {
        const cleanupLoader = initializeLoader(model);
        const cleanupNbState = initializeNBState(model);
        model.send({ ctx: 'bridge', kind: 'info', info: 'initialized' });
        return () => {
            cleanupLoader();
            cleanupNbState();
        };
    }
};
''')

    ctx_name = set(('bridge', 'loader', 'nbstate'))
    feedback = True; watch = False; debug = True;
    state = T.Instance(NB, default_value=NB())
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.feedback_dh = display(HTML(''), display_id=True, metadata=skip())
    
    def close(self):
        if self.feedback_dh: self.feedback_dh.update(HTML(''), metadata=skip())
        self.feedback_dh = None
        super().close()

    @property
    def opts(self): return {'feedback': self.feedback, 'watch': self.watch, 'debug': self.debug}

    def _update_notebook_state(self, req_id):
        if self.feedback_dh and not self._loading: self.feedback_dh.update(
            {NBSTATE_MIME: {**self.opts, 'id': req_id, 'update': True}}, raw=True, metadata=skip())
    def update_notebook_state(self, 
            watch:bool|None=None, feedback:bool|None=None, debug:bool|None=None): 
        if watch is not None: self.watch = watch
        if feedback is not None: self.feedback = feedback
        if debug is not None: self.debug = debug
        return self._update_notebook_state(kounter('nbstate'))

    def update(self, timeout: float|None=None):
        if not self.watch:
            self.update_notebook_state()
            self.send({'cmd': 'get_state'}, timeout=timeout)

    async def aupdate(self, timeout: float=20.0):
        if not self.watch:
            self.update_notebook_state()
            await self.asend({'cmd': 'get_state'}, timeout=timeout)
    
    def setup_notebook_state(self):
        self.feedback_dh.update({NBSTATE_MIME: {**self.opts, 'id': kounter('nbstate')}},  # type: ignore
            raw=True, metadata=skip())
    
    def on_info(self, *args, info:str, **kwargs):
        super().on_info(*args, info=info, **kwargs)
        if info == 'initialized':
            self.logger.show('Requesting notebook state...')
            self.setup_notebook_state()
        elif info == 'found existing state observers':
            self.logger.show('Found existing state observer.')
    
    def on_state_update(self, *args, state:str, **kwargs):
        d = (json.loads(state) if state else {})
        self.state = NB(**d)
        # this shouldn't trigger another update
        self.logger.show(  f"State updated type: {d['type']} #cells {len(d['cells'])} @{d['timestamp']}" 
                    # f" watch: {self.watch} feedback: {self.feedback} debug: {self.debug}"
        )


def get_bridge(): 
    def companions_loader(brd: Bridge):
        cc = []
        if 'htmx' in brd.loaded:  # ensure htmx is loaded - loader should inform us
            cc.append(HTMXCommander(logger=brd.logger, show=False))
        return cc
    return Bridge.instance(companions_loader=companions_loader)


In [ ]:
cleanupwidgets('brd')
# get_capturer().unregister()
# OutputCapture.clear_instance()
BridgeBase.clear_instance()

brd = get_bridge()

# Colophon
----


In [1]:
import inspect
import shutil
from pathlib import Path

import bridget
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [2]:
if FC.IN_NOTEBOOK:
    nb_path = '21_nb_state.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)

    for fn in ('js/nbstate.js', ):
        shutil.copyfile(fn, Path(inspect.getfile(bridget)).parent/fn);
